In [1]:
import matplotlib.pyplot as plt
import itertools
import random
import copy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
from operator import itemgetter
from itertools import groupby
%matplotlib inline
import statistics
import math


In [ ]:
def fitness_landscape(organism_column):
    fitness_value_for_organism = 1
    return fitness_value_for_organism

class world():
    def __init__(self, population_size, loci, gene_mean, gene_sd, proportion_asexual, organism_capacity, asex_repl_ratio, sex_repl_ratio, mutation_prob):
        self.population_size = population_size
        self.organism_capacity = organism_capacity
        self.asex_repl_ratio = asex_repl_ratio
        self.sex_repl_ratio = sex_repl_ratio
        self.total_pop_mat = (np.random.normal(gene_mean, gene_sd, (loci, self.population_size))).astype(int)
        self.separator = int(self.population_size*proportion_asexual) # To indicate where the asex ends and sex begins
        self.mutation_prob = mutation_prob
    
    def population_sizes(self, total = False, asex = False, sex = False):
        if total == True:
            return self.total_pop_mat.shape[1]
        if asex ==True:
            return self.asex_pop_mat.shape[1]
        if sex == True:
            return self.sex_pop_mat.shape[1]
        
    def mutation_stage(self, step = 1):
        shape = self.total_pop_mat.shape
        half_liklihood = self.mutation_prob/2, 3
        mutation_matrix = np.random.choice([-step, 0, step], size=shape, p=[half_liklihood, 1-self.mutation_prob,half_liklihood])
        self.total_pop_mat = self.total_pop_mat + mutation_matrix

    def calc_fitness_array(self):
        self.fitness_array = np.apply_along_axis(fitness_landscape, 0, self.total_pop_mat)
    
    def survival_probability(self):
        #For now we make survival probability simply proportionate to
        #the fitness value. This is fine, and moves all the subtlety to the fitness landscape.
        self.calc_fitness_array()
        total_fitness = np.sum(self.fitness_array)
        prop_fitness = np.divide(self.fitness_array, total_fitness)
        return prop_fitness
    
    def survival_stage(self):
        curr_population_index = range(self.population_sizes(total = True))
        prop_fitness = self.survival_prob()
        #The following step may be a serious computational time issue
        survivor_list = np.random.choice(curr_population_index, self.organism_capacity, replace = False, p = prop_fitness)
        self.total_pop_mat = (self.total_pop_mat)[:,survivor_list]
        self.species_sep_index = np.size(np.where(survivor_list<self.species_sep_index))
        
        #For efficiencies sake, if we need it in replication stage, we should consider
        # updating the fitness array here, and so not calculating it in the replication stage
        #but currently replication isn't dependent on fitness
        #self.fitness_array = self.fitness_array[survivor_list]
        
    def gen_sex_asex_matrices(self):
        self.asex_pop_mat = self.total_pop_mat[:, :self.species_sep_index]
        self.sex_pop_mat = self.total_pop_mat[:, self.species_sep_index:]
    
    def asex_replication_stage(self):
        current_organism_index = np.arange(self.species_sep_index)
        next_gen_size = int(self.species_sep_index*self.asex_repl_ratio)
        next_gen_chosen = np.random.choice(current_organism_index, size = next_gen_size, replace = True)
        self.asex_pop_mat = self.asex_pop_mat[:, next_gen_chosen]   
        
    def sexual_replication(self):
        total_sex_pop = self.population_sizes(sex=True)
        current_organism_index = np.arange(total_sex_pop)
        next_gen_size = int(total_sex_pop*self.sex_repl_ratio)
        def chosen_pairs():
            rounds = int(2*next_gen_size/total_sex_pop) + 1
            mating_pairs = []
            for round in rounds:
                np.random.shuffle(current_organism_index)
                pairs = zip(current_organism_index[::2], current_organism_index[1::2])
                mating_pairs += pairs
            return mating_pairs[:next_gen_size]
        chosen_pairs = chosen_pairs()
        def mate_pairs():
            
        
    
    def replication_stage(self):
        self.gen_sex_asex_matrices()
        self.asex_replication_stage()
        
        